In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd
from utils import specificity_score, negative_prediction_value_score, gmean_score, informedness_score
import torch.nn.functional as F

In [64]:
# Load the data
df = pd.read_csv('./data/HTRU_2.csv', header=None)
df.columns = ['IpMean', 'IpDev', 'IpKurt','IpSkew', 'DMMean', 'DMDev', 'DMKurt', 'DMSkew', 'Class']

# Split the data into features and target
X = df.iloc[:, :-1]
y = df.iloc[:, -1]


# Define the number of experiments
n_runs = 100

# Create a ShuffleSplit instance
ss = ShuffleSplit(n_splits=n_runs, train_size=60, test_size=120)


In [65]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 1, kernel_size=3, stride=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=3)
        self.act2 = nn.ReLU()
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.act2(x)
        x = self.flat(x)
        x = self.fc1(x)
        return x

model = SimpleCNN()


In [66]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Define the number of experiments
n_runs = 100

# Initialize lists to store the scores
scores = {
    'accuracy': [],
    'balanced_accuracy': [],
    'recall': [],
    'specificity': [],
    'precision': [],
    'npv': [],
    'gmean': [],
    'informedness': []
}

ss = ShuffleSplit(n_splits=n_runs, train_size=60, test_size=120)

    

In [70]:
X_tensor.shape

torch.Size([17898, 8])

In [71]:
y_tensor.shape

torch.Size([17898])

In [77]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100
batch_size = 32
trainset = TensorDataset(X_tensor, y_tensor)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(trainset, batch_size=batch_size, shuffle=False)

r = 0


loss_fn = nn.BCELoss()

for epoch in range(n_epochs):
    acct = 0
    countt = 0
    for inputs, labels in trainloader:
        # forward, backward, and then weight update

        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        acct += (torch.round(y_pred) == labels).float().sum()
        countt += len(labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if r % 10 ==0:
            print('training', r,loss)
        r+=1
    acct /= countt
    print("Epoch %d: model training accuracy %.2f%%" % (epoch, acct * 100))

    acc = 0
    count = 0
    for inputs, labels in testloader:
        y_pred = model(inputs)
        acc += (torch.round(y_pred) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))




RuntimeError: Given groups=1, weight of size [1, 1, 3], expected input[1, 32, 8] to have 1 channels, but got 32 channels instead